# Анализ изображений

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style()
%matplotlib inline

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

Внаглую украдем трансформацию с лекции, кое-что добавив: так как resnet, которым мы воспользуемся дальше, кушает 3х-канальные изображения, то нам нужно превратить наш ЧБ-датасет в RGB.

In [3]:
import torchvision.transforms as ttrans

train_transform = ttrans.Compose([
    ttrans.Lambda(lambda image: image.convert('RGB')),
    ttrans.Pad(4),
    ttrans.RandomHorizontalFlip(),
    ttrans.RandomCrop(32),
    ttrans.ToTensor()
])

test_transform = ttrans.Compose([
    ttrans.Lambda(lambda image: image.convert('RGB')),
    ttrans.ToTensor()
])

Я пытался скачать CelebA, но он превысил дневной лимит скачиваний ну Google Drive, как говорится в [этом issue](https://github.com/rasbt/stat453-deep-learning-ss21/issues/4). Потом я попробовал с STL10, который по счастливому стечению обстоятельств похож на Cifar идейно, но он обещал скачиваться трое суток. В итоге мой выбор пал на Fashion MNIST, который выглядит дешево и сердито. Не хочется опять превращать ноутбук в кипятильник.

In [4]:
from torchvision.datasets import FashionMNIST

train_dataset = FashionMNIST(
    root='./FashionMNIST/train',
    train=True, 
    transform=train_transform,
    download=True
)

test_dataset = FashionMNIST(
    root='./FashionMNIST/test',
    train=False, 
    transform=test_transform,
    download=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FashionMNIST/train/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/train/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FashionMNIST/train/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/train/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

...

Extracting ./FashionMNIST/test/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/test/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FashionMNIST/test/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/test/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FashionMNIST/test/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/test/FashionMNIST/raw



Ещё одна фича отправляется к нам прямиком из ноутбука с лекции:

In [5]:
from torch.utils.data import DataLoader

dataloaders = {
    'train': DataLoader(
        dataset=train_dataset,
        batch_size=128, 
        shuffle=True
    ),
    'val': DataLoader(
        dataset=test_dataset,
        batch_size=128, 
        shuffle=False
    )
}

Сделаем класс для нашей модельки. Я решил не выпендриваться и взять `resnet50` в качестве основы модели, как и в лекции.

In [6]:
from torchvision.models import resnet50

class FashionClassifier(nn.Module):
    def __init__(self, ouput_dim, **kwargs):
        super(FashionClassifier, self, **kwargs).__init__() 
        self.model = resnet50(pretrained=True)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        self.fc1 = nn.Sequential(
            nn.Linear(1000, 64),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(64, ouput_dim)
        )
    
    def embed(self, x):
        return self.fc1(self.model(x))
    
    def forward(self, x):
        resnet_out = self.embed(x)
        return self.fc2(resnet_out)

In [7]:
# У нас десять классов одежды
# CUDA у меня нету, но ВДРУГ КТО-ТО РЕШИТ ЗАПУСТИТЬ НА НЕЙ
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = FashionClassifier(10)
model = model.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1e-4)

Опять похитим сниппет с лекции:

In [8]:
from tqdm.notebook import tqdm, trange

loss_hist = {'train': [], 'val': []}
acc_hist = {'train': [], 'val': []}

# Важно! В данном примере точность используется для упрощения.
# Никогда не используйте её если у вас несбалансированная выборка
# Возьмите лучше F_score или ROC_AUC (об этом вам потом расскажут)

epochs = 4

# Основной алгоритм обучения
for epoch in trange(epochs, desc='Whole pipeline'):
    
    for phase in ['train', 'val']:
        dataloader = dataloaders[phase]
        
        # Это условие необходимо так как у нас есть слой DropOut
        # И на валидации его принято фиксировать
        if phase == 'train':
            model.train() 
        elif phase == 'val':
            model.eval()
        
        running_loss = 0.
        running_acc = 0.
        
        # Проходимся по набору данных
        for (X_batch, y_batch) in tqdm(dataloader, desc=f'Epoch: {epoch + 1}. Phase: {phase}'):
            # Нормализуем наши данные
            X_batch = X_batch / 255
            X_batch = ttrans.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_batch)
            
            # Переносим на устройство
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Для корректного обучения перед каждым шагом необходимо сбрасывать прошлые ошибки
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                y_pred = model(X_batch)
                
                loss_value = loss_function(y_pred, y_batch)
                y_pred_class = y_pred.argmax(dim=1)
                
                # На обучении мы хотим учиться в зависимости от ошибки
                if phase == 'train':
                    loss_value.backward()
                    optimizer.step()
                   
            # Аггрегируем ошибку и точность
            running_loss += loss_value.item()
            running_acc += (y_pred_class == y_batch.data).float().mean().data.cpu().numpy()
        
        epoch_loss = running_loss / len(dataloader)
        epoch_acc = running_acc / len(dataloader)
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} ', end='')
        
        loss_hist[phase].append(epoch_loss)
        acc_hist[phase].append(epoch_acc)

Whole pipeline:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch: 1. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

train Loss: 1.8446 Acc: 0.3513 

Epoch: 1. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

val Loss: 19.4720 Acc: 0.1061 

...

val Loss: 13.3688 Acc: 0.1109 

Epoch: 4. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

train Loss: 1.4178 Acc: 0.4934 

Epoch: 4. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

val Loss: 14.0200 Acc: 0.1163 

Давайте теперь всё наше чудо отобразим в тензорборде.

In [9]:
from torch.utils.tensorboard import SummaryWriter

with SummaryWriter() as writer:
    for epoch in range(4):
        writer.add_scalar('Loss/train', loss_hist['train'][epoch], epoch)
        writer.add_scalar('Loss/val', loss_hist['val'][epoch], epoch)
        writer.add_scalar('Accuracy/train', acc_hist['train'][epoch], epoch)
        writer.add_scalar('Accuracy/val', acc_hist['val'][epoch], epoch)

А почему он их три штуки делает? Ну да ладно. Вообще это надо было делать по ходу дела, пока моделька училась. Давайте теперь нарисуем эмбеддинги. 

Перед этим надо сделать небольшой хак, потому что pytorch не очень справляется с эмбеддингами в этой версии: [тык на ишью](https://github.com/pytorch/pytorch/issues/30966).

In [10]:
# Hackity hack
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [11]:
train_items = [train_dataset[i] for i in range(1000)]
xs, ys = zip(*train_items)

xs = torch.stack(xs)
features = xs.mean(dim=1).view(-1, 32 * 32)

with SummaryWriter() as writer:
    writer.add_embedding(features, metadata=ys, label_img=xs)

In [12]:
%load_ext tensorboard
%tensorboard --logdir runs

That's all, folks!